In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [ ]:
## data 준비
(train_X, train_y), (test_X, test_y) = tf.keras.datasets.cifar10.load_data()
train_y = train_y[:,0]
test_y = test_y[:,0]

In [ ]:
print( train_X.shape )
print( train_y.shape )

print( test_X.shape )
print( test_y.shape )

In [ ]:
train_y

In [ ]:
np.unique(train_y)

In [ ]:
for i in range(4):
    plt.imshow(train_X[i])
    plt.title(train_y[i])
    plt.show()

In [ ]:
plt.hist(train_y, bins=10, width=0.5)
plt.show()

In [ ]:
## model 정의
model = tf.keras.models.Sequential([
    tf.keras.layers.InputLayer(input_shape=(32, 32, 3)),

    tf.keras.layers.Conv2D(64, 3, padding='same'),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Conv2D(64, 3, padding='same'),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPool2D((2, 2)),

    tf.keras.layers.Conv2D(128, 3, padding='same'),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Conv2D(128, 3, padding='same'),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPool2D((2, 2)),

    tf.keras.layers.Conv2D(256, 3, padding='same'),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Conv2D(256, 3, padding='same'),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPool2D((2, 2)),

    tf.keras.layers.Conv2D(512, 3, padding='same'),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Conv2D(512, 3, padding='same'),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPool2D((2, 2)),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(512),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10),
    tf.keras.layers.Activation('softmax')
])

model.summary()

## loss 정의
loss = tf.keras.losses.SparseCategoricalCrossentropy()

## 최적화 방법 정의
optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001)

model.compile(optimizer=optimizer,
              loss=loss,
              metrics=['accuracy'],
             )

## 가중치 업데이트
history = model.fit(train_X,train_y,epochs=50,batch_size=1024,
                    validation_data = (test_X, test_y),
                    verbose=1)

In [ ]:
plt.plot( history.history['loss'] )
plt.plot( history.history['val_loss'])
plt.show()
plt.plot( history.history['accuracy'] )
plt.plot( history.history['val_accuracy'])
plt.show()

In [ ]:
model.predict(train_X) # why argmax?

In [ ]:
train_pred = model.predict(train_X).argmax(axis=1)
test_pred = model.predict(test_X).argmax(axis=1)

train_acc = np.sum(train_pred == train_y) / train_y.size
test_acc = np.sum(test_pred == test_y) / test_y.size


test_conf = confusion_matrix(test_y,test_pred)
train_conf = confusion_matrix(train_y,train_pred)


print("train acc:" , train_acc)
print("train confusion matrix\n", train_conf)

print("test acc:" , test_acc)
print("test confusion matrix\n", test_conf)

In [ ]:
disp = ConfusionMatrixDisplay(confusion_matrix=train_conf)
disp.plot()

disp = ConfusionMatrixDisplay(confusion_matrix=test_conf)
disp.plot()

In [ ]:
idx = np.where( (test_y == 4) & (test_y != test_pred) )[0]
print(idx)
for (count,i) in enumerate(idx):
    if count == 10:
        break
    plt.imshow(test_X[i], cmap=plt.cm.gray)
    plt.title("predicted: " + str(test_pred[i]))
    plt.show()